In [ ]:
import numpy as np
import pandas as pd
from dotenv import load_dotenv

from xcontest.xcontest import login, flight_days
from wetter.wetter import load_raw_dataframes,get_wind_data,get_rain_data

load_dotenv()

FLUGBAHN_DIR = 30 # 03 & 21

timeframe_start,timeframe_end = 2019010100,2023123123

df_wind = get_wind_data(timeframe_start,timeframe_end , 13)
df_rain = get_rain_data(timeframe_start,timeframe_end , 13)
df_wind = df_wind.join(df_rain.set_index("MESS_DATUM"), on="MESS_DATUM", validate="1:1" )


df_wind['wind_alignment'] = df_wind.apply(lambda row: np.sin((row['direction'] - FLUGBAHN_DIR )* np.pi / 180.)**2 , axis=1)

print(df_wind.head())

driver = login()

flight_day_list= flight_days(driver, 'Metzingen')

In [56]:
from datetime import datetime
from sklearn.neighbors import KNeighborsRegressor

flight_day_as_mess_date = [int(datetime.strftime(datetime.strptime(d,'%d.%m.%Y'), '%Y%m%d13')) for d in flight_day_list]

df_wind['is_flight_day'] = df_wind.apply(lambda row: 1 if row['MESS_DATUM'] in flight_day_as_mess_date else 0, axis=1)

X, y = df_wind[['strength','wind_alignment','precip']].to_numpy(), df_wind['is_flight_day'].to_numpy() # features and labels
#print(X.shape, y.shape)

# perceptron ?  https://en.wikipedia.org/wiki/Perceptron - no separating hyperplane
# kNN ?
neigh = KNeighborsRegressor(n_neighbors=5, weights= 'distance')
neigh.fit(X, y)

#ytilde = neigh.predict(X)
df_wind['knn'] =  df_wind.apply(lambda row:  neigh.predict([[row['strength'],row['wind_alignment'],row['precip']]]), axis=1)

df_actual = df_wind[(df_wind['is_flight_day']==1)].copy()
df_missed = df_wind[(df_wind['knn']>= 0.4) & (df_wind['is_flight_day']==0)].copy()

In [68]:
df_actual['day'] = df_actual.apply(lambda row: datetime.strptime( str(row['MESS_DATUM'])[:8], "%Y%m%d").strftime('%a'), axis=1 )
print(pd.DataFrame([{k:len(df_actual[(df_actual['day'] == k)]) for k in ['Sat','Sun'] }]))
df_actual[(df_actual['day'] != 'Sun') & (df_actual['day'] != 'Sat')].tail(50) # most flying days are on Sat & Sun

   Sat  Sun
0   19   22


,MESS_DATUM,strength,direction,precip,wind_alignment,is_flight_day,knn,day
606485,2019032913,2.5,240,0.0,0.250000,1,[1.0],Fri
606725,2019040813,3.1,90,0.0,0.750000,1,[0.3333333333333333],Mon
606773,2019041013,3.8,70,0.0,0.413176,1,[0.5],Wed
609989,2019082213,4.5,200,0.0,0.030154,1,[1.0],Thu
610013,2019082313,1.9,240,0.0,0.250000,1,[1.0],Fri
610181,2019083013,1.8,270,0.0,0.750000,1,[0.5],Fri
611669,2019103113,3.5,90,0.0,0.750000,1,[1.0],Thu
613157,2020010113,4.3,160,0.0,0.586824,1,[0.25],Wed
616373,2020051413,3.8,330,0.0,0.750000,1,[0.5],Thu
616541,2020052113,2.7,110,0.0,0.969846,1,[0.5],Thu


In [62]:
df_missed['day'] = df_missed.apply(lambda row: datetime.strptime( str(row['MESS_DATUM'])[:8], "%Y%m%d").strftime('%a'), axis=1 )
df_missed.head(50)

,MESS_DATUM,strength,direction,precip,wind_alignment,is_flight_day,knn,day
604949,2019012413,1.2,60,0.0,0.250000,0,[0.5],Thu
606677,2019040613,4.2,80,0.0,0.586824,0,[0.5],Sat
608981,2019071113,1.8,270,0.0,0.750000,0,[0.5],Thu
610133,2019082813,2.5,250,0.0,0.413176,0,[0.5],Wed
613085,2019122913,3.6,170,0.0,0.413176,0,[0.5],Sun
616277,2020051013,3.9,200,0.0,0.030154,0,[0.5],Sun
619373,2020091613,3.8,330,0.0,0.750000,0,[0.5],Wed
619661,2020092813,3.2,190,0.0,0.116978,0,[0.5],Mon
624317,2021041013,4.9,80,0.0,0.586824,0,[0.5],Sat
625493,2021052913,2.2,320,0.0,0.883022,0,[0.5],Sat
